In [1]:
print("hello world")

hello world


In [2]:
import pandas as pd
import numpy as np

In [5]:
df_funda_2020 = pd.read_csv(r'C:\Users\User\Google Drive\Master DDB\AI for Business\product3team2\data\raw\funda_2020_sold_houses.csv')

In [6]:
df_funda_2020.head()

Error: KeyboardInterrupt

In [8]:
df_funda_2020.dtypes

zipcode                 object
sellingPrice             int64
Asking_Price_M2        float64
Facilities              object
Cadastre_Title          object
Ownership situation     object
fulldescription         object
housetype               object
parcelsurface           object
yearofbuilding          object
description_garden      object
garden_binary            int64
numberrooms              int64
numberbathrooms          int64
energylabelclass        object
surface                  int64
sales_agent             object
buying_agent            object
sellingDate             object
publicationDate         object
sellingtime              int64
url                     object
dtype: object

In [19]:
print (df_funda_2020['buying_agent'].isnull())

0         False
1          True
2          True
3         False
4          True
          ...  
217165     True
217166     True
217167     True
217168     True
217169     True
Name: buying_agent, Length: 217170, dtype: bool


In [26]:
df_funda_2020.fillna("Missing",inplace=True)

In [27]:
df_funda_2020.head(50)

,zipcode,sellingPrice,Asking_Price_M2,Facilities,Cadastre_Title,Ownership situation,fulldescription,housetype,parcelsurface,yearofbuilding,...,numberrooms,numberbathrooms,energylabelclass,surface,sales_agent,buying_agent,sellingDate,publicationDate,sellingtime,url
0,3417EK,315000,2943.925234,Electricity,MONTFOORT A 5048,Full ownership,Deze gunstig gelegen tussenwoning is twee jaar...,"Single-family home, row house\r\n",107 m²,1982,...,5,1,C,107,Beumer Garantiemakelaars,Verschoor Makelaardij & Assurantiën BV,2020-12-02,2020-11-27,5,https://www.funda.nl/en/koop/verkocht/montfoor...
1,3862KT,335000,1340.000000,None,NIJKERK M 635,Full ownership,In een rustige woonwijk met zeer veel groen ge...,"Single-family home, corner house\r\n",250 m²,1976,...,4,1,C,250,Van de Ridder Makelaars,Missing,2020-12-02,2020-11-19,13,https://www.funda.nl/en/koop/verkocht/nijkerk/...
2,6852RJ,395000,2801.418440,Electricity,HUISSEN G 2103,Full ownership,Tribunusallee 39 te Huissen Modern afgewerkte...,"Single-family home, corner house\r\n",141 m²,2010,...,5,2,A,141,Hermsen Garantiemakelaars Huissen,Missing,2020-12-01,2020-11-19,12,https://www.funda.nl/en/koop/verkocht/huissen/...
3,1431HZ,375000,2314.814815,Electricity,AALSMEER C 2721,Full ownership,Would you like to live in one of the nicest st...,"Single-family home, row house\r\n",162 m²,1941,...,5,1,F,162,Eveleens Makelaars,Peters en Partners Amstelveen,2020-12-01,2020-11-21,10,https://www.funda.nl/en/koop/verkocht/aalsmeer...
4,2548HV,425000,3320.312500,Electricity,'S-GRAVENHAGE AX 8313,Municipal ownership encumbered with long-term ...,Zeer fraai en luxueus afgewerkte 5 kamer eenge...,"Single-family home, row house\r\n",128 m²,2008,...,4,1,A,128,Olsthoorn NVM Makelaars,Missing,2020-12-03,2020-11-24,9,https://www.funda.nl/en/koop/verkocht/den-haag...
5,4196JW,469000,1340.000000,Electricity,GELDERMALSEN K 751,Full ownership,Deze keurig verzorgde en uitgebouwde twee-onde...,"Single-family home, double house\r\n",350 m²,1993,...,5,1,A,350,Gerssen & Donkersloot Makelaars,Missing,2020-12-03,2020-11-23,10,https://www.funda.nl/en/koop/verkocht/tricht/h...
6,4041ED,925000,5285.714286,"Electricity, heating and running water",KESTEREN C 342,Full ownership,Ten zuiden van het dorp Kesteren ligt in het b...,"Country house, detached residential property\r\n","13,175 m²",1948,...,7,2,D,175,Wim Tuit - Stad en Land Makelaardij,Missing,2020-11-27,2020-11-20,7,https://www.funda.nl/en/koop/verkocht/kesteren...
7,5481HT,265000,2208.333333,Electricity,SCHIJNDEL H 4305,Full ownership,JONG GELUK! Deze ruime tussenwoning met bergin...,"Single-family home, row house\r\n",120 m²,1977,...,5,1,B,120,Meierijstad Makelaardij,Missing,2020-11-30,2020-11-25,5,https://www.funda.nl/en/koop/verkocht/schijnde...
8,6713GL,240000,1509.433962,Electricity,EDE K 15943,Full ownership,Compleet gerenoveerde MIDDENWONING met stenen ...,"Single-family home, row house\r\n",159 m²,1953,...,4,1,D,159,Ditters Makelaars Ede,Missing,2020-12-02,2020-11-17,15,https://www.funda.nl/en/koop/verkocht/ede/huis...
9,8261GM,160000,3809.523810,None,KAMPEN F 13407,Full ownership,Mooie starterswoning aan de stadsgracht in Kam...,"Single-family home, semi-detached residential ...",42 m²,Before 1906,...,3,1,E,42,Alex woon- & bedrijfsmakelaars,Missing,2020-12-08,2020-11-17,21,https://www.funda.nl/en/koop/verkocht/kampen/h...


In [28]:
df_funda_2020 = pd.read_csv(r'C:\Users\User\Google Drive\Master DDB\AI for Business\product3team2\data\raw\funda_2020_sold_houses.csv')

In [29]:
df_funda_2020.isna().sum()

zipcode                     0
sellingPrice                0
Asking_Price_M2             0
Facilities                  0
Cadastre_Title              0
Ownership situation         0
fulldescription             0
housetype                   0
parcelsurface               0
yearofbuilding              0
description_garden          0
garden_binary               0
numberrooms                 0
numberbathrooms             0
energylabelclass            0
surface                     0
sales_agent                 0
buying_agent           182251
sellingDate                 0
publicationDate             0
sellingtime                 0
url                         0
dtype: int64

In [32]:
df_funda_2020.replace(to_replace="None", value=np.nan, inplace=True)

In [33]:
df_funda_2020.isnull().sum()

zipcode                     7
sellingPrice                0
Asking_Price_M2             0
Facilities              92964
Cadastre_Title          13015
Ownership situation     53580
fulldescription             0
housetype                   0
parcelsurface               0
yearofbuilding              0
description_garden      31328
garden_binary               0
numberrooms                 0
numberbathrooms             0
energylabelclass        10324
surface                     0
sales_agent                 0
buying_agent           182251
sellingDate                 0
publicationDate             0
sellingtime                 0
url                         0
dtype: int64

In [1]:
import pandas as pd
import numpy as np

In [2]:
df_funda_2020 = pd.read_csv(r'C:\Users\User\Google Drive\Master DDB\AI for Business\product3team2\data\raw\funda_2020_sold_houses.csv')

In [3]:
df_funda_2020.replace(to_replace="None", value=np.nan, inplace=True)

In [4]:
df_funda_2020.head(10)

,zipcode,sellingPrice,Asking_Price_M2,Facilities,Cadastre_Title,Ownership situation,fulldescription,housetype,parcelsurface,yearofbuilding,...,numberrooms,numberbathrooms,energylabelclass,surface,sales_agent,buying_agent,sellingDate,publicationDate,sellingtime,url
0,3417EK,315000,2943.925234,Electricity,MONTFOORT A 5048,Full ownership,Deze gunstig gelegen tussenwoning is twee jaar...,"Single-family home, row house\r\n",107 m²,1982,...,5,1,C,107,Beumer Garantiemakelaars,Verschoor Makelaardij & Assurantiën BV,2020-12-02,2020-11-27,5,https://www.funda.nl/en/koop/verkocht/montfoor...
1,3862KT,335000,1340.000000,NaN,NIJKERK M 635,Full ownership,In een rustige woonwijk met zeer veel groen ge...,"Single-family home, corner house\r\n",250 m²,1976,...,4,1,C,250,Van de Ridder Makelaars,NaN,2020-12-02,2020-11-19,13,https://www.funda.nl/en/koop/verkocht/nijkerk/...
2,6852RJ,395000,2801.418440,Electricity,HUISSEN G 2103,Full ownership,Tribunusallee 39 te Huissen Modern afgewerkte...,"Single-family home, corner house\r\n",141 m²,2010,...,5,2,A,141,Hermsen Garantiemakelaars Huissen,NaN,2020-12-01,2020-11-19,12,https://www.funda.nl/en/koop/verkocht/huissen/...
3,1431HZ,375000,2314.814815,Electricity,AALSMEER C 2721,Full ownership,Would you like to live in one of the nicest st...,"Single-family home, row house\r\n",162 m²,1941,...,5,1,F,162,Eveleens Makelaars,Peters en Partners Amstelveen,2020-12-01,2020-11-21,10,https://www.funda.nl/en/koop/verkocht/aalsmeer...
4,2548HV,425000,3320.312500,Electricity,'S-GRAVENHAGE AX 8313,Municipal ownership encumbered with long-term ...,Zeer fraai en luxueus afgewerkte 5 kamer eenge...,"Single-family home, row house\r\n",128 m²,2008,...,4,1,A,128,Olsthoorn NVM Makelaars,NaN,2020-12-03,2020-11-24,9,https://www.funda.nl/en/koop/verkocht/den-haag...
5,4196JW,469000,1340.000000,Electricity,GELDERMALSEN K 751,Full ownership,Deze keurig verzorgde en uitgebouwde twee-onde...,"Single-family home, double house\r\n",350 m²,1993,...,5,1,A,350,Gerssen & Donkersloot Makelaars,NaN,2020-12-03,2020-11-23,10,https://www.funda.nl/en/koop/verkocht/tricht/h...
6,4041ED,925000,5285.714286,"Electricity, heating and running water",KESTEREN C 342,Full ownership,Ten zuiden van het dorp Kesteren ligt in het b...,"Country house, detached residential property\r\n","13,175 m²",1948,...,7,2,D,175,Wim Tuit - Stad en Land Makelaardij,NaN,2020-11-27,2020-11-20,7,https://www.funda.nl/en/koop/verkocht/kesteren...
7,5481HT,265000,2208.333333,Electricity,SCHIJNDEL H 4305,Full ownership,JONG GELUK! Deze ruime tussenwoning met bergin...,"Single-family home, row house\r\n",120 m²,1977,...,5,1,B,120,Meierijstad Makelaardij,NaN,2020-11-30,2020-11-25,5,https://www.funda.nl/en/koop/verkocht/schijnde...
8,6713GL,240000,1509.433962,Electricity,EDE K 15943,Full ownership,Compleet gerenoveerde MIDDENWONING met stenen ...,"Single-family home, row house\r\n",159 m²,1953,...,4,1,D,159,Ditters Makelaars Ede,NaN,2020-12-02,2020-11-17,15,https://www.funda.nl/en/koop/verkocht/ede/huis...
9,8261GM,160000,3809.523810,NaN,KAMPEN F 13407,Full ownership,Mooie starterswoning aan de stadsgracht in Kam...,"Single-family home, semi-detached residential ...",42 m²,Before 1906,...,3,1,E,42,Alex woon- & bedrijfsmakelaars,NaN,2020-12-08,2020-11-17,21,https://www.funda.nl/en/koop/verkocht/kampen/h...


In [5]:
print("All "None" values relaced by "NaN")


SyntaxError: invalid syntax (<ipython-input-5-ab29271e5406>, line 1)

In [6]:
print("All None values relaced by NaN")

All None values relaced by NaN


In [12]:
df_funda_2020.drop(axis=1, columns='Ownership situation')

,zipcode,sellingPrice,Asking_Price_M2,Facilities,Cadastre_Title,fulldescription,housetype,parcelsurface,yearofbuilding,description_garden,...,numberrooms,numberbathrooms,energylabelclass,surface,sales_agent,buying_agent,sellingDate,publicationDate,sellingtime,url
0,3417EK,315000,2943.925234,Electricity,MONTFOORT A 5048,Deze gunstig gelegen tussenwoning is twee jaar...,"Single-family home, row house\r\n",107 m²,1982,Back garden,...,5,1,C,107,Beumer Garantiemakelaars,Verschoor Makelaardij & Assurantiën BV,2020-12-02,2020-11-27,5,https://www.funda.nl/en/koop/verkocht/montfoor...
1,3862KT,335000,1340.000000,NaN,NIJKERK M 635,In een rustige woonwijk met zeer veel groen ge...,"Single-family home, corner house\r\n",250 m²,1976,Back garden and side garden,...,4,1,C,250,Van de Ridder Makelaars,NaN,2020-12-02,2020-11-19,13,https://www.funda.nl/en/koop/verkocht/nijkerk/...
2,6852RJ,395000,2801.418440,Electricity,HUISSEN G 2103,Tribunusallee 39 te Huissen Modern afgewerkte...,"Single-family home, corner house\r\n",141 m²,2010,Back garden,...,5,2,A,141,Hermsen Garantiemakelaars Huissen,NaN,2020-12-01,2020-11-19,12,https://www.funda.nl/en/koop/verkocht/huissen/...
3,1431HZ,375000,2314.814815,Electricity,AALSMEER C 2721,Would you like to live in one of the nicest st...,"Single-family home, row house\r\n",162 m²,1941,Back garden and front garden,...,5,1,F,162,Eveleens Makelaars,Peters en Partners Amstelveen,2020-12-01,2020-11-21,10,https://www.funda.nl/en/koop/verkocht/aalsmeer...
4,2548HV,425000,3320.312500,Electricity,'S-GRAVENHAGE AX 8313,Zeer fraai en luxueus afgewerkte 5 kamer eenge...,"Single-family home, row house\r\n",128 m²,2008,Back garden and front garden,...,4,1,A,128,Olsthoorn NVM Makelaars,NaN,2020-12-03,2020-11-24,9,https://www.funda.nl/en/koop/verkocht/den-haag...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
217165,5612AB,459000,2467.741935,NaN,NaN,BUNKERTOREN EINDHOVEN: Schrijf je nu in! Ga na...,Apartment with shared street entrance (apartme...,186 m³,2019,NaN,...,2,1,A,186,vb&t Nieuwbouwmakelaars,NaN,2019-12-10,2019-12-06,4,https://www.funda.nl/en/koop/verkocht/eindhove...
217166,4364AK,115000,440.613027,NaN,MARIEKERKE K 856,"Het betreft hier een ""kopen naar wens"" woning....","Single-family home, double house\r\n",261 m²,1947,NaN,...,4,2,D,261,Sinke Komejan Makelaars,NaN,2019-11-29,2019-11-29,0,https://www.funda.nl/en/koop/verkocht/grijpske...
217167,5612AB,354500,1704.326923,NaN,NaN,Modern wonen op een historische plek in Eindho...,Apartment with shared street entrance (apartme...,208 m³,2019,NaN,...,3,1,A,208,vb&t Nieuwbouwmakelaars,NaN,2019-12-10,2019-12-06,4,https://www.funda.nl/en/koop/verkocht/eindhove...
217168,3994ZC,415000,2577.639752,NaN,HOUTEN A 8467,direct verkocht,"Single-family home, row house\r\n",161 m²,1998,NaN,...,1,1,B,161,Brecheisen Makelaars Houten B.V.,NaN,2019-12-10,2019-12-10,0,https://www.funda.nl/en/koop/verkocht/houten/h...


In [13]:
df_funda_2020['parcelsurface'] = df_funda_2020['parcelsurface'].str.replace(r'\D', '').astype(int)

In [14]:
df_funda_2020.head(10)

,zipcode,sellingPrice,Asking_Price_M2,Facilities,Cadastre_Title,Ownership situation,fulldescription,housetype,parcelsurface,yearofbuilding,...,numberrooms,numberbathrooms,energylabelclass,surface,sales_agent,buying_agent,sellingDate,publicationDate,sellingtime,url
0,3417EK,315000,2943.925234,Electricity,MONTFOORT A 5048,Full ownership,Deze gunstig gelegen tussenwoning is twee jaar...,"Single-family home, row house\r\n",107,1982,...,5,1,C,107,Beumer Garantiemakelaars,Verschoor Makelaardij & Assurantiën BV,2020-12-02,2020-11-27,5,https://www.funda.nl/en/koop/verkocht/montfoor...
1,3862KT,335000,1340.000000,NaN,NIJKERK M 635,Full ownership,In een rustige woonwijk met zeer veel groen ge...,"Single-family home, corner house\r\n",250,1976,...,4,1,C,250,Van de Ridder Makelaars,NaN,2020-12-02,2020-11-19,13,https://www.funda.nl/en/koop/verkocht/nijkerk/...
2,6852RJ,395000,2801.418440,Electricity,HUISSEN G 2103,Full ownership,Tribunusallee 39 te Huissen Modern afgewerkte...,"Single-family home, corner house\r\n",141,2010,...,5,2,A,141,Hermsen Garantiemakelaars Huissen,NaN,2020-12-01,2020-11-19,12,https://www.funda.nl/en/koop/verkocht/huissen/...
3,1431HZ,375000,2314.814815,Electricity,AALSMEER C 2721,Full ownership,Would you like to live in one of the nicest st...,"Single-family home, row house\r\n",162,1941,...,5,1,F,162,Eveleens Makelaars,Peters en Partners Amstelveen,2020-12-01,2020-11-21,10,https://www.funda.nl/en/koop/verkocht/aalsmeer...
4,2548HV,425000,3320.312500,Electricity,'S-GRAVENHAGE AX 8313,Municipal ownership encumbered with long-term ...,Zeer fraai en luxueus afgewerkte 5 kamer eenge...,"Single-family home, row house\r\n",128,2008,...,4,1,A,128,Olsthoorn NVM Makelaars,NaN,2020-12-03,2020-11-24,9,https://www.funda.nl/en/koop/verkocht/den-haag...
5,4196JW,469000,1340.000000,Electricity,GELDERMALSEN K 751,Full ownership,Deze keurig verzorgde en uitgebouwde twee-onde...,"Single-family home, double house\r\n",350,1993,...,5,1,A,350,Gerssen & Donkersloot Makelaars,NaN,2020-12-03,2020-11-23,10,https://www.funda.nl/en/koop/verkocht/tricht/h...
6,4041ED,925000,5285.714286,"Electricity, heating and running water",KESTEREN C 342,Full ownership,Ten zuiden van het dorp Kesteren ligt in het b...,"Country house, detached residential property\r\n",13175,1948,...,7,2,D,175,Wim Tuit - Stad en Land Makelaardij,NaN,2020-11-27,2020-11-20,7,https://www.funda.nl/en/koop/verkocht/kesteren...
7,5481HT,265000,2208.333333,Electricity,SCHIJNDEL H 4305,Full ownership,JONG GELUK! Deze ruime tussenwoning met bergin...,"Single-family home, row house\r\n",120,1977,...,5,1,B,120,Meierijstad Makelaardij,NaN,2020-11-30,2020-11-25,5,https://www.funda.nl/en/koop/verkocht/schijnde...
8,6713GL,240000,1509.433962,Electricity,EDE K 15943,Full ownership,Compleet gerenoveerde MIDDENWONING met stenen ...,"Single-family home, row house\r\n",159,1953,...,4,1,D,159,Ditters Makelaars Ede,NaN,2020-12-02,2020-11-17,15,https://www.funda.nl/en/koop/verkocht/ede/huis...
9,8261GM,160000,3809.523810,NaN,KAMPEN F 13407,Full ownership,Mooie starterswoning aan de stadsgracht in Kam...,"Single-family home, semi-detached residential ...",42,Before 1906,...,3,1,E,42,Alex woon- & bedrijfsmakelaars,NaN,2020-12-08,2020-11-17,21,https://www.funda.nl/en/koop/verkocht/kampen/h...


In [15]:
df_funda_2020['yearofbuilding'] = df_funda_2020['yearofbuilding'].str.replace(r'\D', '')

In [16]:
df_funda_2020.head(10)

,zipcode,sellingPrice,Asking_Price_M2,Facilities,Cadastre_Title,Ownership situation,fulldescription,housetype,parcelsurface,yearofbuilding,...,numberrooms,numberbathrooms,energylabelclass,surface,sales_agent,buying_agent,sellingDate,publicationDate,sellingtime,url
0,3417EK,315000,2943.925234,Electricity,MONTFOORT A 5048,Full ownership,Deze gunstig gelegen tussenwoning is twee jaar...,"Single-family home, row house\r\n",107,1982,...,5,1,C,107,Beumer Garantiemakelaars,Verschoor Makelaardij & Assurantiën BV,2020-12-02,2020-11-27,5,https://www.funda.nl/en/koop/verkocht/montfoor...
1,3862KT,335000,1340.000000,NaN,NIJKERK M 635,Full ownership,In een rustige woonwijk met zeer veel groen ge...,"Single-family home, corner house\r\n",250,1976,...,4,1,C,250,Van de Ridder Makelaars,NaN,2020-12-02,2020-11-19,13,https://www.funda.nl/en/koop/verkocht/nijkerk/...
2,6852RJ,395000,2801.418440,Electricity,HUISSEN G 2103,Full ownership,Tribunusallee 39 te Huissen Modern afgewerkte...,"Single-family home, corner house\r\n",141,2010,...,5,2,A,141,Hermsen Garantiemakelaars Huissen,NaN,2020-12-01,2020-11-19,12,https://www.funda.nl/en/koop/verkocht/huissen/...
3,1431HZ,375000,2314.814815,Electricity,AALSMEER C 2721,Full ownership,Would you like to live in one of the nicest st...,"Single-family home, row house\r\n",162,1941,...,5,1,F,162,Eveleens Makelaars,Peters en Partners Amstelveen,2020-12-01,2020-11-21,10,https://www.funda.nl/en/koop/verkocht/aalsmeer...
4,2548HV,425000,3320.312500,Electricity,'S-GRAVENHAGE AX 8313,Municipal ownership encumbered with long-term ...,Zeer fraai en luxueus afgewerkte 5 kamer eenge...,"Single-family home, row house\r\n",128,2008,...,4,1,A,128,Olsthoorn NVM Makelaars,NaN,2020-12-03,2020-11-24,9,https://www.funda.nl/en/koop/verkocht/den-haag...
5,4196JW,469000,1340.000000,Electricity,GELDERMALSEN K 751,Full ownership,Deze keurig verzorgde en uitgebouwde twee-onde...,"Single-family home, double house\r\n",350,1993,...,5,1,A,350,Gerssen & Donkersloot Makelaars,NaN,2020-12-03,2020-11-23,10,https://www.funda.nl/en/koop/verkocht/tricht/h...
6,4041ED,925000,5285.714286,"Electricity, heating and running water",KESTEREN C 342,Full ownership,Ten zuiden van het dorp Kesteren ligt in het b...,"Country house, detached residential property\r\n",13175,1948,...,7,2,D,175,Wim Tuit - Stad en Land Makelaardij,NaN,2020-11-27,2020-11-20,7,https://www.funda.nl/en/koop/verkocht/kesteren...
7,5481HT,265000,2208.333333,Electricity,SCHIJNDEL H 4305,Full ownership,JONG GELUK! Deze ruime tussenwoning met bergin...,"Single-family home, row house\r\n",120,1977,...,5,1,B,120,Meierijstad Makelaardij,NaN,2020-11-30,2020-11-25,5,https://www.funda.nl/en/koop/verkocht/schijnde...
8,6713GL,240000,1509.433962,Electricity,EDE K 15943,Full ownership,Compleet gerenoveerde MIDDENWONING met stenen ...,"Single-family home, row house\r\n",159,1953,...,4,1,D,159,Ditters Makelaars Ede,NaN,2020-12-02,2020-11-17,15,https://www.funda.nl/en/koop/verkocht/ede/huis...
9,8261GM,160000,3809.523810,NaN,KAMPEN F 13407,Full ownership,Mooie starterswoning aan de stadsgracht in Kam...,"Single-family home, semi-detached residential ...",42,1906,...,3,1,E,42,Alex woon- & bedrijfsmakelaars,NaN,2020-12-08,2020-11-17,21,https://www.funda.nl/en/koop/verkocht/kampen/h...


In [35]:
df_funda_2020 = pd.read_csv(r'C:\Users\User\Google Drive\Master DDB\AI for Business\product3team2\data\raw\funda_2020_sold_houses.csv')

In [37]:
df_funda_2020['yearofbuilding'].str.contains("-")

0         False
1         False
2         False
3         False
4         False
          ...  
217165    False
217166    False
217167    False
217168    False
217169    False
Name: yearofbuilding, Length: 217170, dtype: bool

In [56]:
df_funda_2020['yearofbuilding'].apply(lambda x: mean_year(x))

0         1982
1         1976
2         2010
3         1941
4         2008
          ... 
217165    2019
217166    1947
217167    2019
217168    1998
217169    2019
Name: yearofbuilding, Length: 217170, dtype: int64

In [55]:
year = "1980-1991"
def mean_year(x):
    x = x.replace('After ', '')
    x = x.replace('Before ', '')
    if "-" in x:
        x = x.split("-")
        x = (int(x[0]) + int(x[1])) / 2
        return(int(x))
    else:
        return int(x)

mean_year(year)

1985

In [57]:
def mean_year(x):
    x = x.replace('After ', '')
    x = x.replace('Before ', '')
    if "-" in x:
        x = x.split("-")
        x = (int(x[0]) + int(x[1])) / 2
        return(int(x))
    else:
        return int(x)

df_funda_2020['yearofbuilding'].apply(lambda x: mean_year(x))

0         1982
1         1976
2         2010
3         1941
4         2008
          ... 
217165    2019
217166    1947
217167    2019
217168    1998
217169    2019
Name: yearofbuilding, Length: 217170, dtype: int64

In [83]:
def mean_year(date):
    date = date.replace('After ', '')
    date = date.replace('Before ', '')
    if "-" in date:
        date = date.split("-")
        date = (int(date[0]) + int(date[1])) / 2
        return(int(date))
    else:
        return int(date)

df_funda_2020['yearofbuilding'].apply(lambda date: mean_year(date))

0         1982
1         1976
2         2010
3         1941
4         2008
          ... 
217165    2019
217166    1947
217167    2019
217168    1998
217169    2019
Name: yearofbuilding, Length: 217170, dtype: int64

In [60]:
class DataCleaner(self, object):
    def load_funda_2020(self):
        df_funda_2020 = pd.read_csv(r'C:\Users\User\Google Drive\Master DDB\AI for Business\product3team2\data\raw\funda_2020_sold_houses.csv')
        return df_funda_2020

NameError: name 'self' is not defined

In [61]:
class DataCleaner(object):
    def load_funda_2020(self):
        df_funda_2020 = pd.read_csv(r'C:\Users\User\Google Drive\Master DDB\AI for Business\product3team2\data\raw\funda_2020_sold_houses.csv')
        return df_funda_2020

In [62]:
    def replace_None_types(self, data):    
        df_funda_2020.replace(to_replace="None", value=np.nan, inplace=True)
        print("All None values relaced by NaN")
        return data 

In [63]:
    def replace_None_types(self, data):    
        df_funda_2020.replace(to_replace="None", value=np.nan, inplace=True)
        print("All None values relaced by NaN")
        return df_funda_2020

In [64]:
class DataCleaner(object):
    def load_funda_2020(self):
        df_funda_2020 = pd.read_csv(r'C:\Users\User\Google Drive\Master DDB\AI for Business\product3team2\data\raw\funda_2020_sold_houses.csv')
        return df_funda_2020

    def replace_None_types(self, data):    
        df_funda_2020.replace(to_replace="None", value=np.nan, inplace=True)
        print("All None values relaced by NaN")
        return df_funda_2020 
        
    def drop_ownership_situation(self, data):
        df_funda_2020.drop(axis=1, columns='Ownership situation')
        print("Column Ownership situation dropped")
        return df_funda_2020     
        
    def remove_m3_parcelsurface(self, data):
        df_funda_2020['parcelsurface'] = df_funda_2020['parcelsurface'].str.replace(r'\D', '').astype(int)
        print("Unit m³ removed from parcelsurface")
        return df_funda_2020

    def mean_year(self, date):
        date = date.replace('After ', '') # replace 'After ' with empty
        date = date.replace('Before ', '') # replace 'Before ' with empty
        if "-" in date: # check whether there is a '-' in the column
            date = date.split("-") # if yes, split the two dates
            date = (int(date[0]) + int(date[1])) / 2 # calculate the mean of the two dates
            return(int(date)) # return the mean data
        else:
            return int(date) # if not, return the only date

        # apply function to each row of the column 
        df_funda_2020['yearofbuilding'].apply(lambda date: self.mean_year(date))
        print("Column yearofbuilding ready for calculations")

In [65]:
df_funda_2020.head(60)

,zipcode,sellingPrice,Asking_Price_M2,Facilities,Cadastre_Title,Ownership situation,fulldescription,housetype,parcelsurface,yearofbuilding,...,numberrooms,numberbathrooms,energylabelclass,surface,sales_agent,buying_agent,sellingDate,publicationDate,sellingtime,url
0,3417EK,315000,2943.925234,Electricity,MONTFOORT A 5048,Full ownership,Deze gunstig gelegen tussenwoning is twee jaar...,"Single-family home, row house\r\n",107 m²,1982,...,5,1,C,107,Beumer Garantiemakelaars,Verschoor Makelaardij & Assurantiën BV,2020-12-02,2020-11-27,5,https://www.funda.nl/en/koop/verkocht/montfoor...
1,3862KT,335000,1340.000000,None,NIJKERK M 635,Full ownership,In een rustige woonwijk met zeer veel groen ge...,"Single-family home, corner house\r\n",250 m²,1976,...,4,1,C,250,Van de Ridder Makelaars,NaN,2020-12-02,2020-11-19,13,https://www.funda.nl/en/koop/verkocht/nijkerk/...
2,6852RJ,395000,2801.418440,Electricity,HUISSEN G 2103,Full ownership,Tribunusallee 39 te Huissen Modern afgewerkte...,"Single-family home, corner house\r\n",141 m²,2010,...,5,2,A,141,Hermsen Garantiemakelaars Huissen,NaN,2020-12-01,2020-11-19,12,https://www.funda.nl/en/koop/verkocht/huissen/...
3,1431HZ,375000,2314.814815,Electricity,AALSMEER C 2721,Full ownership,Would you like to live in one of the nicest st...,"Single-family home, row house\r\n",162 m²,1941,...,5,1,F,162,Eveleens Makelaars,Peters en Partners Amstelveen,2020-12-01,2020-11-21,10,https://www.funda.nl/en/koop/verkocht/aalsmeer...
4,2548HV,425000,3320.312500,Electricity,'S-GRAVENHAGE AX 8313,Municipal ownership encumbered with long-term ...,Zeer fraai en luxueus afgewerkte 5 kamer eenge...,"Single-family home, row house\r\n",128 m²,2008,...,4,1,A,128,Olsthoorn NVM Makelaars,NaN,2020-12-03,2020-11-24,9,https://www.funda.nl/en/koop/verkocht/den-haag...
5,4196JW,469000,1340.000000,Electricity,GELDERMALSEN K 751,Full ownership,Deze keurig verzorgde en uitgebouwde twee-onde...,"Single-family home, double house\r\n",350 m²,1993,...,5,1,A,350,Gerssen & Donkersloot Makelaars,NaN,2020-12-03,2020-11-23,10,https://www.funda.nl/en/koop/verkocht/tricht/h...
6,4041ED,925000,5285.714286,"Electricity, heating and running water",KESTEREN C 342,Full ownership,Ten zuiden van het dorp Kesteren ligt in het b...,"Country house, detached residential property\r\n","13,175 m²",1948,...,7,2,D,175,Wim Tuit - Stad en Land Makelaardij,NaN,2020-11-27,2020-11-20,7,https://www.funda.nl/en/koop/verkocht/kesteren...
7,5481HT,265000,2208.333333,Electricity,SCHIJNDEL H 4305,Full ownership,JONG GELUK! Deze ruime tussenwoning met bergin...,"Single-family home, row house\r\n",120 m²,1977,...,5,1,B,120,Meierijstad Makelaardij,NaN,2020-11-30,2020-11-25,5,https://www.funda.nl/en/koop/verkocht/schijnde...
8,6713GL,240000,1509.433962,Electricity,EDE K 15943,Full ownership,Compleet gerenoveerde MIDDENWONING met stenen ...,"Single-family home, row house\r\n",159 m²,1953,...,4,1,D,159,Ditters Makelaars Ede,NaN,2020-12-02,2020-11-17,15,https://www.funda.nl/en/koop/verkocht/ede/huis...
9,8261GM,160000,3809.523810,None,KAMPEN F 13407,Full ownership,Mooie starterswoning aan de stadsgracht in Kam...,"Single-family home, semi-detached residential ...",42 m²,Before 1906,...,3,1,E,42,Alex woon- & bedrijfsmakelaars,NaN,2020-12-08,2020-11-17,21,https://www.funda.nl/en/koop/verkocht/kampen/h...


In [66]:
class DataCleaner(object):
    def load_funda_2020(self):
        df_funda_2020 = pd.read_csv(r'C:\Users\User\Google Drive\Master DDB\AI for Business\product3team2\data\raw\funda_2020_sold_houses.csv')
        return df_funda_2020

    def replace_None_types(self, data):    
        df_funda_2020.replace(to_replace="None", value=np.nan, inplace=True)
        print("All None values relaced by NaN")
        return df_funda_2020 

In [67]:
df_funda_2020.head(50)

,zipcode,sellingPrice,Asking_Price_M2,Facilities,Cadastre_Title,Ownership situation,fulldescription,housetype,parcelsurface,yearofbuilding,...,numberrooms,numberbathrooms,energylabelclass,surface,sales_agent,buying_agent,sellingDate,publicationDate,sellingtime,url
0,3417EK,315000,2943.925234,Electricity,MONTFOORT A 5048,Full ownership,Deze gunstig gelegen tussenwoning is twee jaar...,"Single-family home, row house\r\n",107 m²,1982,...,5,1,C,107,Beumer Garantiemakelaars,Verschoor Makelaardij & Assurantiën BV,2020-12-02,2020-11-27,5,https://www.funda.nl/en/koop/verkocht/montfoor...
1,3862KT,335000,1340.000000,None,NIJKERK M 635,Full ownership,In een rustige woonwijk met zeer veel groen ge...,"Single-family home, corner house\r\n",250 m²,1976,...,4,1,C,250,Van de Ridder Makelaars,NaN,2020-12-02,2020-11-19,13,https://www.funda.nl/en/koop/verkocht/nijkerk/...
2,6852RJ,395000,2801.418440,Electricity,HUISSEN G 2103,Full ownership,Tribunusallee 39 te Huissen Modern afgewerkte...,"Single-family home, corner house\r\n",141 m²,2010,...,5,2,A,141,Hermsen Garantiemakelaars Huissen,NaN,2020-12-01,2020-11-19,12,https://www.funda.nl/en/koop/verkocht/huissen/...
3,1431HZ,375000,2314.814815,Electricity,AALSMEER C 2721,Full ownership,Would you like to live in one of the nicest st...,"Single-family home, row house\r\n",162 m²,1941,...,5,1,F,162,Eveleens Makelaars,Peters en Partners Amstelveen,2020-12-01,2020-11-21,10,https://www.funda.nl/en/koop/verkocht/aalsmeer...
4,2548HV,425000,3320.312500,Electricity,'S-GRAVENHAGE AX 8313,Municipal ownership encumbered with long-term ...,Zeer fraai en luxueus afgewerkte 5 kamer eenge...,"Single-family home, row house\r\n",128 m²,2008,...,4,1,A,128,Olsthoorn NVM Makelaars,NaN,2020-12-03,2020-11-24,9,https://www.funda.nl/en/koop/verkocht/den-haag...
5,4196JW,469000,1340.000000,Electricity,GELDERMALSEN K 751,Full ownership,Deze keurig verzorgde en uitgebouwde twee-onde...,"Single-family home, double house\r\n",350 m²,1993,...,5,1,A,350,Gerssen & Donkersloot Makelaars,NaN,2020-12-03,2020-11-23,10,https://www.funda.nl/en/koop/verkocht/tricht/h...
6,4041ED,925000,5285.714286,"Electricity, heating and running water",KESTEREN C 342,Full ownership,Ten zuiden van het dorp Kesteren ligt in het b...,"Country house, detached residential property\r\n","13,175 m²",1948,...,7,2,D,175,Wim Tuit - Stad en Land Makelaardij,NaN,2020-11-27,2020-11-20,7,https://www.funda.nl/en/koop/verkocht/kesteren...
7,5481HT,265000,2208.333333,Electricity,SCHIJNDEL H 4305,Full ownership,JONG GELUK! Deze ruime tussenwoning met bergin...,"Single-family home, row house\r\n",120 m²,1977,...,5,1,B,120,Meierijstad Makelaardij,NaN,2020-11-30,2020-11-25,5,https://www.funda.nl/en/koop/verkocht/schijnde...
8,6713GL,240000,1509.433962,Electricity,EDE K 15943,Full ownership,Compleet gerenoveerde MIDDENWONING met stenen ...,"Single-family home, row house\r\n",159 m²,1953,...,4,1,D,159,Ditters Makelaars Ede,NaN,2020-12-02,2020-11-17,15,https://www.funda.nl/en/koop/verkocht/ede/huis...
9,8261GM,160000,3809.523810,None,KAMPEN F 13407,Full ownership,Mooie starterswoning aan de stadsgracht in Kam...,"Single-family home, semi-detached residential ...",42 m²,Before 1906,...,3,1,E,42,Alex woon- & bedrijfsmakelaars,NaN,2020-12-08,2020-11-17,21,https://www.funda.nl/en/koop/verkocht/kampen/h...


In [68]:
    def replace_None_types(self, data):    
        df_funda_2020.replace(to_replace="None", value=np.nan, inplace=True)
        print("All None values relaced by NaN")
        return df_funda_2020 

In [69]:
df_funda_2020.head(50)

,zipcode,sellingPrice,Asking_Price_M2,Facilities,Cadastre_Title,Ownership situation,fulldescription,housetype,parcelsurface,yearofbuilding,...,numberrooms,numberbathrooms,energylabelclass,surface,sales_agent,buying_agent,sellingDate,publicationDate,sellingtime,url
0,3417EK,315000,2943.925234,Electricity,MONTFOORT A 5048,Full ownership,Deze gunstig gelegen tussenwoning is twee jaar...,"Single-family home, row house\r\n",107 m²,1982,...,5,1,C,107,Beumer Garantiemakelaars,Verschoor Makelaardij & Assurantiën BV,2020-12-02,2020-11-27,5,https://www.funda.nl/en/koop/verkocht/montfoor...
1,3862KT,335000,1340.000000,None,NIJKERK M 635,Full ownership,In een rustige woonwijk met zeer veel groen ge...,"Single-family home, corner house\r\n",250 m²,1976,...,4,1,C,250,Van de Ridder Makelaars,NaN,2020-12-02,2020-11-19,13,https://www.funda.nl/en/koop/verkocht/nijkerk/...
2,6852RJ,395000,2801.418440,Electricity,HUISSEN G 2103,Full ownership,Tribunusallee 39 te Huissen Modern afgewerkte...,"Single-family home, corner house\r\n",141 m²,2010,...,5,2,A,141,Hermsen Garantiemakelaars Huissen,NaN,2020-12-01,2020-11-19,12,https://www.funda.nl/en/koop/verkocht/huissen/...
3,1431HZ,375000,2314.814815,Electricity,AALSMEER C 2721,Full ownership,Would you like to live in one of the nicest st...,"Single-family home, row house\r\n",162 m²,1941,...,5,1,F,162,Eveleens Makelaars,Peters en Partners Amstelveen,2020-12-01,2020-11-21,10,https://www.funda.nl/en/koop/verkocht/aalsmeer...
4,2548HV,425000,3320.312500,Electricity,'S-GRAVENHAGE AX 8313,Municipal ownership encumbered with long-term ...,Zeer fraai en luxueus afgewerkte 5 kamer eenge...,"Single-family home, row house\r\n",128 m²,2008,...,4,1,A,128,Olsthoorn NVM Makelaars,NaN,2020-12-03,2020-11-24,9,https://www.funda.nl/en/koop/verkocht/den-haag...
5,4196JW,469000,1340.000000,Electricity,GELDERMALSEN K 751,Full ownership,Deze keurig verzorgde en uitgebouwde twee-onde...,"Single-family home, double house\r\n",350 m²,1993,...,5,1,A,350,Gerssen & Donkersloot Makelaars,NaN,2020-12-03,2020-11-23,10,https://www.funda.nl/en/koop/verkocht/tricht/h...
6,4041ED,925000,5285.714286,"Electricity, heating and running water",KESTEREN C 342,Full ownership,Ten zuiden van het dorp Kesteren ligt in het b...,"Country house, detached residential property\r\n","13,175 m²",1948,...,7,2,D,175,Wim Tuit - Stad en Land Makelaardij,NaN,2020-11-27,2020-11-20,7,https://www.funda.nl/en/koop/verkocht/kesteren...
7,5481HT,265000,2208.333333,Electricity,SCHIJNDEL H 4305,Full ownership,JONG GELUK! Deze ruime tussenwoning met bergin...,"Single-family home, row house\r\n",120 m²,1977,...,5,1,B,120,Meierijstad Makelaardij,NaN,2020-11-30,2020-11-25,5,https://www.funda.nl/en/koop/verkocht/schijnde...
8,6713GL,240000,1509.433962,Electricity,EDE K 15943,Full ownership,Compleet gerenoveerde MIDDENWONING met stenen ...,"Single-family home, row house\r\n",159 m²,1953,...,4,1,D,159,Ditters Makelaars Ede,NaN,2020-12-02,2020-11-17,15,https://www.funda.nl/en/koop/verkocht/ede/huis...
9,8261GM,160000,3809.523810,None,KAMPEN F 13407,Full ownership,Mooie starterswoning aan de stadsgracht in Kam...,"Single-family home, semi-detached residential ...",42 m²,Before 1906,...,3,1,E,42,Alex woon- & bedrijfsmakelaars,NaN,2020-12-08,2020-11-17,21,https://www.funda.nl/en/koop/verkocht/kampen/h...


In [78]:
from funda.felicia.py import DataCleaner

ModuleNotFoundError: No module named 'funda'

ModuleNotFoundError: No module named 'product3team2'

In [82]:
class DataCleaner(object):
    def clean_funda_2020(data):
        df_funda_2020 = pd.read_csv(r'C:\Users\User\Google Drive\Master DDB\AI for Business\product3team2\data\raw\funda_2020_sold_houses.csv')
        df_funda_2020.replace(to_replace="None", value=np.nan, inplace=True)
        print("All None values relaced by NaN")
        df_funda_2020.drop(axis=1, columns='Ownership situation')
        print("Column Ownership situation dropped")    
        df_funda_2020['parcelsurface'] = df_funda_2020['parcelsurface'].str.replace(r'\D', '').astype(int)
        print("Unit m³ removed from parcelsurface")
        return df_funda_2020

    def mean_year(self, date):
        date = date.replace('After ', '') # replace 'After ' with empty
        date = date.replace('Before ', '') # replace 'Before ' with empty
        if "-" in date: # check whether there is a '-' in the column
            date = date.split("-") # if yes, split the two dates
            date = (int(date[0]) + int(date[1])) / 2 # calculate the mean of the two dates
            return(int(date)) # return the mean data
        else:
            return int(date) # if not, return the only date

        # apply function to each row of the column 
        df_funda_2020['yearofbuilding'].apply(lambda date: self.mean_year(date))
        print("Column yearofbuilding ready for calculations")